In [ ]:
import numpy as np

from pydrake.all import (
    FindResourceOrThrow,
    Parser,
    AddMultibodyPlantSceneGraph,
    ConnectMeshcatVisualizer,
    DiagramBuilder,
    JacobianWrtVariable,
    Simulator,
)

import multibody_extras as me

In [ ]:
def normalize(x):
    norm = np.linalg.norm(x)
    eps = np.sqrt(np.finfo(float).eps)
    assert norm > eps
    return x / norm

In [ ]:
np.set_printoptions(formatter={"float_kind": lambda x: f"{x:.4f}"})

In [ ]:
def make_parser(plant):
    parser = Parser(plant)
    parser.package_map().PopulateFromFolder("./repos/universal_robot")
    return parser

In [ ]:
# %%script bash --bg
# # Launch a drake-visualizer instance in the background.
# drake-visualizer

# # TODO(eric): Embed meshcat.

In [ ]:
# model_file = FindResourceOrThrow("drake/manipulation/models/franka_description/urdf/panda_arm.urdf")
# q = np.array([0.03, -0.53, -0.04, -2.3, 0.01, 2.25, -0.81])
# v_limit = np.deg2rad([85, 85, 100, 75, 130, 135, 135])

# model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/urdf/iiwa14_no_collision.urdf")
# v_limit = np.deg2rad([85, 85, 100, 75, 130, 135, 135])
# q = np.array([0.03, 0.53, -0.04, -1.3, 0.01, -0.3, -0.81])

model_file = "./repos/universal_robot/ur_description/urdf/ur5.urdf"
v_limit = np.deg2rad([85, 85, 100, 75, 130, 135])
q = np.array([0.03, 0.53, -0.04, -1.3, 0.01, -0.3])

In [ ]:
# Load arm; ensure we can visualize.
builder = DiagramBuilder()
arbitrary_plant_time_step = 0.01  # See: https://github.com/RobotLocomotion/drake/issues/14688
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=arbitrary_plant_time_step)

arm = make_parser(plant).AddModelFromFile(model_file)
arm_bodies = me.get_bodies(plant, {arm})
# Naively select base and ee body.
base_body = arm_bodies[0]
ee_body = arm_bodies[-1]

frame_W = plant.world_frame()
frame_B = base_body.body_frame()
frame_G = ee_body.body_frame()
# Weld it (get rid of floating-dof stuff)
plant.WeldFrames(frame_W, frame_B)
plant.Finalize()

# DrakeVisualizer.AddToBuilder(builder, scene_graph)
meshcat_vis = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url="new", open_browser=False)

diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
context = plant.GetMyContextFromRoot(diagram_context)

display(meshcat_vis.vis.jupyter_cell())
Simulator(diagram, diagram_context.Clone()).Initialize()

In [ ]:
plant.SetPositions(context, arm, q)
diagram.Publish(diagram_context)

In [ ]:
# # Meh. These seem to lie when coming from an URDF, at least the ones in Drake.
# v_lower = plant.GetVelocitiesFromArray(arm, plant.GetVelocityLowerLimits())
# v_upper = plant.GetVelocitiesFromArray(arm, plant.GetVelocityUpperLimits())
# # Show that limits are symmetric.
# v_limit = v_upper
# assert np.all(np.abs(v_lower) == v_limit)

In [ ]:
# Compute Jacobian at present configuration.
J_WG = plant.CalcJacobianSpatialVelocity(
    context,
    with_respect_to=JacobianWrtVariable.kV,
    frame_B=frame_G,
    p_BP=[0, 0, 0],
    frame_A=frame_W,
    frame_E=frame_W,
)
# Check pinv solutions for only the translation slice.
# (Lets angular velocity be unconstrained)
Jp_WG = J_WG[3:]

In [ ]:
# Specify target EE linear velocity norm.
pdot_norm = 2.0  # m/s

# Choose some random directions for naive empirical analysis.
pdot_units = np.array([normalize(x) for x in [
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [1.0, 1.0, 0.0],
    [1.0, 0.0, 1.0],
    [0.0, 1.0, 1.0],
    [1.0, 1.0, 1.0],
]])

In [ ]:
# Naive linear solve + iteration.
vs = []

for pdot_unit in pdot_units:
    pdot_WG = pdot_norm * pdot_unit
    v = np.linalg.lstsq(Jp_WG, pdot_WG)[0]
    vs.append(v)

vs = np.array(vs)
print(vs)

In [ ]:
exceeds_limits = vs > v_limit
print(exceeds_limits.astype(np.uint8))